<a href="https://colab.research.google.com/github/labnac/scuolaAIP/blob/main/2_keras_code.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import tensorflow as tf
from keras.models import Sequential
from keras.layers import Dense
from sklearn.datasets import load_wine
from tensorflow.keras.datasets import mnist
import tensorflow.keras
from tensorflow.keras import layers
import numpy as np
import matplotlib.pyplot as plt
from numpy import genfromtxt
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OneHotEncoder, StandardScaler


def create_custom_model(input_dim, output_dim, nodes, layers=1, name='model'):
    # Create model
    model = Sequential(name=name)
    for i in range(layers):
      if i == 1:
        model.add(Dense(nodes[i], input_dim=input_dim, activation='linear'))
      else:
        model.add(Dense(nodes[i], input_dim=input_dim, activation='sigmoid'))
    model.add(Dense(output_dim, activation='linear'))

    # Compile model
    model.compile(loss='mean_squared_error',
                  optimizer='adam',
                  metrics=['RootMeanSquaredError'])
    return model

wine = load_wine()
features = wine['data']
target = wine['target']
names = wine['target_names']
feature_names = wine['feature_names']
enc = OneHotEncoder()
target = enc.fit_transform(target[:, np.newaxis]).toarray()

# Scale data to have mean 0 and variance 1
# which is importance for convergence of the neural network
scaler = StandardScaler()
features_scaled = scaler.fit_transform(features)
# Split the data set into training and testing
X_train, X_test, Y_train, Y_test = train_test_split(
    features_scaled, target, test_size=0.2, random_state=2)

n_features = features.shape[1]
n_classes = target.shape[1]



n_layers = 1  # must be greater than 0
n_hidden_neurons = 8
model = create_custom_model(n_features, n_classes, n_hidden_neurons, n_layers)

model.summary()

from keras.callbacks import TensorBoard

history_dict = {}

# TensorBoard Callback
cb = TensorBoard()


history_callback = model.fit(X_train, Y_train,
                              batch_size=32,
                              epochs=200,
                              verbose=1,
                              validation_data=(X_test, Y_test),
                              callbacks=[cb])
score = model.evaluate(X_test, Y_test, verbose=0)
print('Test loss:', score[0])
print('Test accuracy:', score[1])
print(history_callback)
history_dict[model.name] = [history_callback, model]

fig, (ax1, ax2) = plt.subplots(2, figsize=(8, 6))

for model_name in history_dict:
    val_accuracy = history_dict[model_name][0].history['val_accuracy']
    val_loss = history_dict[model_name][0].history['val_loss']
    accuracy = history_dict[model_name][0].history['accuracy']
    loss = history_dict[model_name][0].history['loss']
    ax1.plot(val_accuracy, label=model_name + " val_accuracy")
    ax1.plot(accuracy, label=model_name + " accuracy")
    ax2.plot(val_loss, label=model_name + " val loss")
    ax2.plot(loss, label=model_name + " loss")

ax1.set_ylabel('validation accuracy')
ax2.set_ylabel('validation loss')
ax2.set_xlabel('epochs')
ax1.legend()
ax2.legend()
plt.show()

